## Import

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

import pathlib
from typing import Any,Union

2022-09-23 14:22:32.201179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 14:22:32.479803: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-23 14:22:33.347190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-23 14:22:33.347238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

## Def

In [2]:
BATCH_SIZE:int  = 64
IMG_HEIGHT:int  = 227
IMG_WIDTH:int   = 227

# Valdiate 20% 
VAL_SPLIT:float = 0.2

EPOCHS=2

In [3]:
def get_dataset_dir(path) -> str:
    return pathlib.Path(path)

In [4]:
def load_dataset(dir) -> Union[list | Any, list | Any]:
  train_ds = tf.keras.utils.image_dataset_from_directory(
    dir,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)
  val_ds = tf.keras.utils.image_dataset_from_directory(
    dir,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)
  return train_ds, val_ds

## Load Dataset

In [5]:
data_dir:str = get_dataset_dir('giga_data')
print("Nombres d'image: {}".format(len(list(data_dir.glob('*/*.jpg')))))


train_ds, val_ds = load_dataset(data_dir)
class_names = train_ds.class_names

Nombres d'image: 40195
Found 40995 files belonging to 2 classes.
Using 32796 files for training.


2022-09-23 14:22:35.864625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 14:22:35.915489: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 14:22:35.915612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-23 14:22:35.917315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 40995 files belonging to 2 classes.
Using 8199 files for validation.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

# Load datasets in cache to optimize 
train_ds = train_ds.cache().shuffle(10).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model

In [7]:
print(tf.config.experimental.list_physical_devices())
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [8]:
with tf.device('/GPU:0'):
    num_classes = len(class_names)

    # Def model
    model = tf.keras.Sequential([
        # tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        #tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMG_HEIGHT,IMG_WIDTH,3)),

        tf.keras.layers.Conv2D(32, 3, 2, activation='relu'),
        tf.keras.layers.MaxPooling2D(strides=2),

        tf.keras.layers.Conv2D(32, 3, 2, activation='relu'),
        tf.keras.layers.MaxPooling2D(strides=2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])


    # Compile model
    model.compile(optimizer='adamax', # or adam
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])

## Train

In [9]:
with tf.device('/GPU:0'):
    history:tf.keras.callbacks.History = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

Epoch 1/2


2022-09-23 14:22:40.319297: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-23 14:22:40.959991: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


333/513 [==================>...........] - ETA: 13s - loss: 1.4861 - accuracy: 0.7431

2022-09-23 14:23:06.072600: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


344/513 [===================>..........] - ETA: 12s - loss: 1.4512 - accuracy: 0.7441

Corrupt JPEG data: 419 extraneous bytes before marker 0xd9


512/513 [============================>.] - ETA: 0s - loss: 1.1040 - accuracy: 0.7520

: 

: 

## Validate

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Validate ++

In [ ]:
# from glob import glob

# def testAndPrint(model_):
#     validation_path = pathlib.Path("giga_test")
#     class_names = list(glob('*',root_dir='giga_test'))
#     print(class_names)

#     isPhotoError = 0
#     miss = {}
#     for class_name in class_names:
#         miss[class_name] = 0

#     for class_name in class_names:
#         err = 0
#         sum_perc = 0
#         notdef = 0

#         plt.figure(figsize=(100, 100))
#         for class_name_2 in class_names:
#             miss[class_name_2] = 0
        
#         for path in list(validation_path.glob(f"{class_name}/*.jpg")):
#             img = tf.keras.utils.load_img(
#                 path, target_size=(IMG_HEIGHT, IMG_WIDTH)
#             )
#             img_array = tf.keras.utils.img_to_array(img)
#             img_array = tf.expand_dims(img_array, 0) # Create a batch

#             # predictions = model_.predict(img_array,verbose=0)
#             score = model_.predict(img_array,verbose=0)

#             # score = tf.nn.softmax(predictions[0])
#             #miss[class_names[np.argmax(score)]] += 1
#             #if 100 * np.max(score) < 80:
#             #        notdef = notdef + 1
#             #else:
#             # print(np.argmax(score))

#             if (class_name == 'photo' and score < 0.5) or (class_name != 'photo' and score > 0.5):
#             # if (class_name == 'Photo' and np.argmax(score) > 0.5) or (class_name != 'Photo' and np.argmax(score) < 0.5):
#                 isPhotoError +=1
#                 #print(
#                 #    "{} {} with a {:.2f} percent confidence but is a {}"
#                 #    .format(path, class_names[np.argmax(score)], 100 * np.max(score), class_name)
#                 #)

#                 err = err + 1
#                 sum_perc = sum_perc + 100 * np.max(score)
#                 # if err <= 40:
#                     # plt.subplot(5, 8, err)
#                     # plt.imshow(img)
#                     # plt.title(class_names[np.argmax(score)],fontsize=60)
#                     # plt.axis("off")
                
#                 miss['Error'] += 1
#             else:
#                 miss['Right'] += 1


#         len_class = len(list(validation_path.glob(f"{class_name}/*.jpg")))
#         if err > 0:
#             stat = ((len_class-err-notdef) / len_class) * 100
#             print(f"{class_name} {stat}% || {err} error || {notdef} nodef || {sum_perc/err}")
#         else:
#             stat = ((len_class-notdef) / len_class) * 100
#             print(f"{class_name} {stat}% || {notdef} nodef")

#         print(f"Results for {class_name}: {miss}")
        
#     print(f"Number of errors: {isPhotoError}")
#     plt.show()


In [ ]:
# with tf.device('/GPU:0'):
#     testAndPrint(model)

## Save

In [ ]:
import datetime
import pandas as pd

date:datetime = datetime.datetime.now().strftime("%m.%d.%Y_%H:%M:%S")
model_dir:str = f"models/model_{date}"
model.save(model_dir)
f = open(f"{model_dir}/model_summary.txt", "a")
model.summary(print_fn=lambda x: f.write(x + '\n'))
f.close()

f = open(f"{model_dir}/model_history.csv", "a")
f.write(pd.DataFrame.from_dict(history.history).to_csv(index=False))
f.close()

In [ ]:
# Plot test image, predicted label, and the true label
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# Plot predictions bars, color correct predictions in blue and incorrect predictions in red
def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Correct prediction
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()